<a href="https://colab.research.google.com/github/SikoticVinyl/VictorVis2.0/blob/aaron_halo-analytics/Halo_Stats_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install Required Packages
%%capture
!pip install requests pandas matplotlib seaborn python-dotenv

In [ ]:
#Import Libraries
import os
from datetime import datetime, timedelta
import time
from pathlib import Path
import json
import logging
from functools import lru_cache
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files

In [ ]:
API Key Setup
# Create this cell to handle API key securely in Colab
from google.colab import userdata
#Upload .env file
uploaded = files.upload()  # Upload your .env file
from dotenv import load_dotenv
load_dotenv()

In [ ]:
#HaloAPIClient Class Definition
class HaloAPIClient:
    """Client for interacting with the Halo Stats API."""

    BASE_URL = "https://www.haloapi.com/stats"
    RATE_LIMIT = 10  # requests per second

    def __init__(self, cache_dir: str = "/content/cache"):
        """Initialize the API client."""
        self.api_key = os.getenv("HALO_API_KEY")
        if not self.api_key:
            raise ValueError("HALO_API_KEY environment variable not set")

        self.session = requests.Session()
        self.session.headers.update({
            "Ocp-Apim-Subscription-Key": self.api_key
        })

        self.cache_dir = Path(cache_dir)
        self.cache_dir.mkdir(exist_ok=True)

        self.last_request_time = 0
        self.logger = logging.getLogger(__name__)

    def _rate_limit(self):
        """Implement rate limiting."""
        current_time = time.time()
        time_since_last = current_time - self.last_request_time
        if time_since_last < (1.0 / self.RATE_LIMIT):
            time.sleep((1.0 / self.RATE_LIMIT) - time_since_last)
        self.last_request_time = time.time()

    def _cached_get(self, endpoint: str, params: dict = None) -> dict:
        """Make a GET request with caching."""
        cache_key = f"{endpoint}_{str(params)}.json"
        cache_file = self.cache_dir / cache_key

        if cache_file.exists():
            cache_age = time.time() - cache_file.stat().st_mtime
            if cache_age < 3600:  # Cache for 1 hour
                return json.loads(cache_file.read_text())

        self._rate_limit()
        response = self.session.get(f"{self.BASE_URL}{endpoint}", params=params)
        response.raise_for_status()
        data = response.json()

        cache_file.write_text(json.dumps(data))
        return data

In [ ]:
#API Methods
def get_match_history(self, player: str, count: int = 25) -> pd.DataFrame:
    """Get a player's match history."""
    data = self._cached_get(
        f"/h5/players/{player}/matches",
        params={"count": count}
    )

    matches = []
    for match in data.get("Results", []):
        match_data = {
            "match_id": match["Id"]["MatchId"],
            "game_mode": match["Id"]["GameMode"],
            "playlist_id": match.get("HopperId"),
            "map_id": match["MapId"],
            "duration": match["MatchDuration"],
            "date": match["MatchCompletedDate"]["ISO8601Date"]
        }

        player_stats = match["Players"][0]
        match_data.update({
            "rank": player_stats["Rank"],
            "result": player_stats["Result"],
            "kills": player_stats.get("TotalKills", 0),
            "deaths": player_stats.get("TotalDeaths", 0),
            "assists": player_stats.get("TotalAssists", 0)
        })

        matches.append(match_data)

    return pd.DataFrame(matches)

def get_service_record(self, player: str) -> dict:
    """Get a player's arena service record."""
    data = self._cached_get(
        "/h5/servicerecords/arena",
        params={"players": player}
    )
    return data["Results"][0]["Result"]

def get_match_details(self, match_id: str) -> dict:
    """Get detailed stats for a specific match."""
    return self._cached_get(f"/h5/arena/matches/{match_id}")

In [ ]:
#Analysis Methods
def analyze_player_performance(self, player: str, matches: int = 50) -> pd.DataFrame:
    """Analyze a player's performance trends."""
    match_history = self.get_match_history(player, matches)

    match_history["kd_ratio"] = match_history["kills"] / match_history["deaths"]
    match_history["kda_ratio"] = (match_history["kills"] + match_history["assists"]) / match_history["deaths"]
    match_history["date"] = pd.to_datetime(match_history["date"])

    match_history["kd_rolling_avg"] = match_history["kd_ratio"].rolling(window=5).mean()
    match_history["kills_rolling_avg"] = match_history["kills"].rolling(window=5).mean()

    return match_history

def plot_performance_trends(self, player: str, metric: str = "kd_ratio"):
    """Plot performance trends for a player."""
    performance = self.analyze_player_performance(player)

    plt.figure(figsize=(12, 6))
    sns.set_style("darkgrid")

    sns.lineplot(data=performance, x="date", y=metric)

    rolling_col = f"{metric}_rolling_avg"
    if rolling_col in performance.columns:
        sns.lineplot(data=performance, x="date", y=rolling_col, color="red", label="5-game average")

    plt.title(f"{player} - {metric.replace('_', ' ').title()} Over Time")
    plt.xticks(rotation=45)
    plt.tight_layout()

    return plt

In [ ]:
#HaloStatsAnalyzer Class
class HaloStatsAnalyzer:
    """Helper class for analyzing Halo stats."""

    def __init__(self, api_client: HaloAPIClient):
        """Initialize the analyzer."""
        self.client = api_client

    def compare_players(self, players: list[str], metric: str = "kd_ratio") -> pd.DataFrame:
        """Compare stats between multiple players."""
        all_stats = []

        for player in players:
            stats = self.client.analyze_player_performance(player)
            stats["player"] = player
            all_stats.append(stats)

        return pd.concat(all_stats)

    def plot_player_comparison(self, players: list[str], metric: str = "kd_ratio"):
        """Create a visualization comparing players."""
        comparison_data = self.compare_players(players, metric)

        plt.figure(figsize=(12, 6))
        sns.boxplot(data=comparison_data, x="player", y=metric)
        plt.title(f"Player Comparison - {metric.replace('_', ' ').title()}")
        plt.xticks(rotation=45)
        plt.tight_layout()

        return plt

In [ ]:
#Usage Example
# Initialize the client
client = HaloAPIClient()

# Example usage - replace with actual gamertag
PLAYER_GAMERTAG = "YourGamertag"  # Replace with actual gamertag

# Get match history
matches = client.get_match_history(PLAYER_GAMERTAG)
print("Recent Matches:")
display(matches.head())

# Analyze performance
performance = client.analyze_player_performance(PLAYER_GAMERTAG)
print("\nPerformance Analysis:")
display(performance.describe())

# Create visualization
plt = client.plot_performance_trends(PLAYER_GAMERTAG)
plt.show()

In [ ]:
Player Comparison Example
# Initialize analyzer
analyzer = HaloStatsAnalyzer(client)

# Compare multiple players
PLAYERS = ["Player1", "Player2", "Player3"]  # Replace with actual gamertags
comparison = analyzer.compare_players(PLAYERS)
print("Player Comparison:")
display(comparison.groupby("player").mean())

# Create comparison visualization
plt = analyzer.plot_player_comparison(PLAYERS)
plt.show()